# imports and setup

In [ ]:
%%capture
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
!pip install "xformers<0.0.26" trl peft accelerate bitsandbytes pinecone-client sentence-transformers langchain-openai openai

In [ ]:
import torch
from datasets import load_dataset
from huggingface_hub import notebook_login
from transformers import TrainingArguments
from trl import SFTTrainer
from unsloth import FastLanguageModel
from transformers import AutoTokenizer
import numpy as np

In [ ]:
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/llama-3-8b-Instruct-bnb-4bit",
    max_seq_length = 2048,
    dtype = torch.float16,
    load_in_4bit = True,
)

model = FastLanguageModel.get_peft_model(
    model,
    r = 256,
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 512,
    lora_dropout = 0,
    bias = "none",
    use_gradient_checkpointing = "unsloth",
    random_state = 3407,
    use_rslora = False,
    loftq_config = None,
)

tokenizer.padding_side = "left"

In [ ]:
from pinecone import Pinecone, ServerlessSpec

pc = Pinecone(api_key='INSERT_PINECONE_API_KEY_HERE')

In [ ]:
index = pc.Index("medmcqa-train-1")

In [ ]:
from langchain_openai import OpenAIEmbeddings
embed_model = OpenAIEmbeddings(model="text-embedding-ada-002", openai_api_key="INSERT_OPEN_API_KEY_HERE")

def get_relevant_context(text, k=1):
  embeds = embed_model.embed_documents([text])
  results_1 = index.query(
      vector=embeds[0],
      top_k=k,
      include_metadata=True
  )

  results = []

  for result in results_1["matches"]:
    results.append(result["metadata"]["text"])
  return results

# load datasets

In [ ]:
medmcqa = load_dataset("openlifescienceai/medmcqa", split="validation")
medmcqa_anatomy = medmcqa.filter(lambda example: example["subject_name"] == "Anatomy")

In [ ]:
print(medmcqa_anatomy)

In [ ]:
FastLanguageModel.for_inference(model)

In [ ]:
def predict(prompt):
    inputs = tokenizer([prompt], return_tensors = "pt").to("cuda")
    outputs = model.generate(**inputs, max_new_tokens = 256, use_cache = True)
    return tokenizer.batch_decode(outputs[:, inputs.input_ids.shape[1]:], skip_special_tokens = True)[0].strip()

In [ ]:
import json

with open('word_defs.json') as file:
  word_defs = json.load(file)

# medmcqa anatomy

In [ ]:
def eval_medmcqa_anatomy():
  references = []
  predictions = []
  SAMPLE_CNT = len(medmcqa_anatomy)
  mismatch_cnt = 0

  for i in range(SAMPLE_CNT):
    example = medmcqa_anatomy[i]
    options = [example['opa'], example['opb'], example['opc'], example['opd']]

    prompt = "<|begin_of_text|><|start_header_id|>user<|end_header_id|>\n\n" + example["question"] + "\n\n" + example['opa'] + "\n" + example['opb'] + "\n" + example['opc'] + "\n" + example['opd'] + "\n\nRespond with the correct choice from the list above verbatim.  Do not include any explanation."

    if all(option in word_defs.keys() for option in options):
      prompt += " You may use the following definitions only if they are helpful: \n"
      for option in options:
        if option in word_defs.keys():
          prompt += f'''{option}: {word_defs[option]}\n'''

    prompt += "<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\n"
    prediction = predict(prompt)

    if prediction not in options:
      prompt += prediction + "<|eot_id|>\n<|start_header_id|>user<|end_header_id|>\n\nYour response does not exactly match one of the choices from the list. Do not apologise or include any text other than one of the options from the list verbatim without any label. Here are the options again\n\n" + example['opa'] + "\n\n" + example['opb'] + "\n\n" + example['opc'] + "\n\n" + example['opd'] + "\n\n<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\n"
      prediction = predict(prompt)

    reference = options[example['cop']]

    if prediction not in options:
      mismatch_cnt += 1

    references.append(reference)
    predictions.append(prediction)

  em = sum([1 if prediction == reference else 0 for prediction, reference in zip(predictions, references)]) / SAMPLE_CNT
  mismatch = mismatch_cnt / SAMPLE_CNT
  return em, mismatch

em, mismatch = eval_medmcqa_anatomy()

In [ ]:
print("exact match:", em)
print("mismatch:", mismatch)

exact match: 0.5854700854700855
mismatch: 0.017094017094017096
